In [7]:
import csv
import requests
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString


In [8]:
cc_xml = ''
with open('./Conversation_SSML.xml', 'r') as cc_file:
    cc_xml = cc_file.read()

In [9]:
conversation = []
i=1

tree = parseString(cc_xml)
for el in tree.getElementsByTagName('voice'):
    conv_part = ''
    speaker = (dict(el.attributes.items())['name'])
    for node in el.childNodes:
        if node.nodeType == node.TEXT_NODE:
            conv_part = node.data
        elif node.tagName == 'prosody':
            conv_part = conv_part + node.firstChild.data
        else: 
            for pro in node.getElementsByTagName('prosody'):
                conv_part = conv_part + pro.firstChild.data
    part = 'Agent'
    if speaker == 'en-US-JennyNeural' : 
        part = 'Agent'
    else:
        part = 'Customer'
    conversation.append({"text":conv_part, 'id':i, 'role':part, 'participantId':'{}'.format(part)})
    i = i+1
    
#print(conversation)


In [10]:
# Format Conversation for Language Service Analysis

json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  "tasks": [             
       {
        "taskName": "analyze 1",
        "kind": "ConversationalSummarizationTask",
        "parameters": {
           "summaryAspects": ["chaptertitle","issue", "resolution", "narrative"]

        }
    }
  ]
}

In [11]:
json_d

{'displayName': 'Conversation Task Example',
 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hello, how may I help you today?',
      'id': 1,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "I really need help with my credit card, it's not working at all",
      'id': 2,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'May I please have your first and last name?',
      'id': 3,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "sure it's John, J O H N, Doh, D O E",
      'id': 4,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'Thank you Mr Doh, can you confirm the last four digits of your account number?',
      'id': 5,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "Which number? Is that the card number or the number on my statement, I don't have a statement in front of me.",
      'id': 6,
      'role': 'Customer',
      'participantId': '

## Open AI Summarization

In [20]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-12-01"
openai.api_key = parser.get('openai_api','api_key')
model = parser.get('openai_api','api_model')

In [25]:
prompt = "Generate detailed call summary notes in list format from the following text\nText:\n###\n{}\n###\n- ".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.2,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:
 Call Summary Notes:
1. Customer John Doh contacted the Agent for help with his credit card.
2. Agent asked for the customer's first and last name, which was provided.
3. Agent asked for the last four digits of the customer's account number, which was provided.
4. Agent informed the customer that there was suspected fraud on his credit card and asked for the last purchase made.
5. Customer confirmed the purchase of an Air Bee En Bee for his daughter in the amount of two thousand dollars.
6. Agent marked the purchase as approved and the customer proceeded with the booking.
7. Customer requested to order another card for his daughter and provided her first and last name.
8. Agent asked for the address to mail the card to, which was provided.
9. Customer was informed that the card would arrive in 1 to 2 business days.
10. No further assistance was required.


In [27]:
prompt = "Extract list of call participants in list format from the following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:
List of call participants:
- Agent
- Customer


In [29]:
prompt = "Generate list of call follow up tasks from following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.7,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

List of call follow up tasks:
1. Follow up with customer to confirm the last four digits of their account number
2. Follow up with customer to confirm the charge amount of their last purchase
3. Follow up with customer to confirm the address to mail the new card to
4. Follow up with customer to confirm receipt of the new card
